In [15]:
import pydicom
from pydicom import dcmread

from sam2.build_sam import build_sam2_video_predictor
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.distributions as td
import os
device = torch.device("cuda")
checkpoint = "//mnt/e/weights/sam2/sam2.1_hiera_large.pt"
model_cfg = "//mnt/e/code/sam2-main/sam2/configs/sam2.1/sam2.1_hiera_l.yaml"


In [355]:
base_path='/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/'
Ground_folder_t1='/T1DUAL/Ground'
dcm_files_in_t1='/T1DUAL/DICOM_anon/InPhase'
dcm_files_out_t1='/T1DUAL/DICOM_anon/OutPhase'

Ground_folder_t2='/T2SPIR/Ground'
dcm_files_t2='/T2SPIR/DICOM_anon/'
organs=[63,126,189,252]

In [91]:
patients=os.listdir('/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/')

In [96]:
patients.sort(key=lambda p: int(p))

# **Some Function** 

In [675]:

def get_mask_all(organ,mask_dir,filenames):
    masks=[]
    for i in range(len(filenames)):
        mask=Image.open(os.path.join(mask_dir, filenames[i]))
        mask=np.asarray(mask)==organs[organ]
        masks.append(mask)
    return np.asarray(masks)

def get_input(mask_dir,filenames):
    IMGS=[]
    for i in range(len(filenames)):
        mask=Image.open(os.path.join(mask_dir, filenames[i]))
        mask=np.asarray(mask)#.convert("RGB")
        IMGS.append(mask)
    return np.asarray(IMGS)

In [470]:
def get_input_frame_all(path_1,name_1,path_2,name_2):
    input_t1_1=[]
    input_t1_2=[]
    for i in range(len(name_1[0])):
        ds_t1_1 = dcmread(os.path.join(path_1[0],name_1[0][i]))
        arr_t1_1 = ds_t1_1.pixel_array
        input_t1_1.append(arr_t1_1)
    print(os.path.join(path_1[0],name_1[0][i]))
    for i in range(len(name_1[1])):
        ds_t1_2 = dcmread(os.path.join(path_1[1],name_1[1][i]))
        arr_t1_2 = ds_t1_2.pixel_array
        input_t1_2.append(arr_t1_2)
    
    print(os.path.join(path_1[1],name_1[1][i]))
    input_t2=[]
    for i in range(len(name_2[0])):
        ds_t1_1 = dcmread(os.path.join(path_2[0],name_2[0][i]))
        arr_t1_1 = ds_t1_1.pixel_array
        input_t2.append(arr_t1_1)

    print(os.path.join(path_2[0],name_2[0][i]))
    return np.asarray(input_t1_1), np.asarray(input_t1_2), np.asarray(input_t2)

In [755]:

def get_contour(mask_):
    contours, _ = cv2.findContours(mask_.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    M = cv2.moments(cnt)    
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return cx,cy

def radius(center,rad,sample_rate):
    x=np.cos(np.linspace(0,2*np.pi-0.001,sample_rate))*rad
    y=np.sin(np.linspace(0,2*np.pi-0.001,sample_rate))*rad
    x=center[0]+x
    y=center[1]+y
    output=np.stack((x,y)).T
    zzz=np.sum(output>0,axis=-1)==2
    return output[np.where(zzz==1)]

def get_means(position,POS,NEG,step):
    means_p=[]
    for i in range(step):
        kkk=radius(position,POS[i],int(POS[i]*5))
        means_p.append(kkk)
    means_n=[]
    for i in range(step):
        kkk=radius(position,NEG[i],int(NEG[i]*5))
        means_n.append(kkk)
    return means_p,means_n

def Get_gmm(mean,var):
    num_of_gaus=len(mean)
    covars=torch.ones((num_of_gaus,2,2))*torch.eye(2)*var
    mean=torch.Tensor(mean)
    MVN= td.MultivariateNormal(mean,covars)
    mix=td.Categorical(torch.ones((num_of_gaus,)))
    gmm = td.MixtureSameFamily(mix, MVN)
    return gmm


def extract_mask(res_,w_,h_):
    mask_output=[]
    for i in range(len(res_)):
        for out_obj_id, out_mask in res_[i].items():
            mask_output.append(out_mask.reshape((w_,h_,1))*1.0)
    mask_output=np.asarray(mask_output).astype(np.float32)
    return mask_output

def TV_dist(img1,img2):
    tv=img2-img1
    tv2=np.sum(np.abs(tv))
    return tv,tv2

def dm_dt_func(all_masks):
    dm_dt_img=[]
    dm_dt_val=[]
    for i in range(len(all_masks)-1):
        k1,k2=TV_dist(all_masks[i],all_masks[i+1])
        dm_dt_img.append(k1)
        dm_dt_val.append(k2)
    dm_dt2_img=[]
    dm_dt2_val=[]
    for i in range(len(dm_dt_img)-1):
        k1,k2=TV_dist(dm_dt_img[i],dm_dt_img[i+1])
        dm_dt2_img.append(k1)
    for i in range(len(dm_dt_val)-1):
        dm_dt2_val.append(np.sum(np.abs(dm_dt_val[i+1]-dm_dt_val[i])))
    return dm_dt_val/np.max(dm_dt_val),dm_dt2_val/np.max(dm_dt2_val)


def dm_dx_dy_func(all_masks):
    TV_dx=[]
    TV_dy=[]
    for i in range(len(all_masks)):
        TV_dx.append(np.sum(np.abs(all_masks[i][1:,:]-all_masks[i][:-1,:])))
        TV_dy.append(np.sum(np.abs(all_masks[i][:,1:]-all_masks[i][:,:-1])))
    TV_dy=np.asarray(TV_dy)
    TV_dx=np.asarray(TV_dx)
    outs=(TV_dx+TV_dy)/2
    return outs/np.max(outs)

def metrcis_func(all_masks):
    dm_dt,dm_dt2=dm_dt_func(all_masks)
    dm_dx_dy=dm_dx_dy_func(all_masks)
    return np.sum(dm_dt),np.sum(dm_dt2),np.sum(dm_dx_dy)

def iou(outputs: np.array, labels: np.array):
    SMOOTH = 1e-6
    intersection = (outputs & labels).sum()
    union = (outputs | labels).sum()
    iou = (intersection ) / (union + SMOOTH)
    return iou

def dice(outputs: np.array, labels: np.array):
    intersection = (outputs * labels).sum()
    union = np.sum(outputs).sum()+np.sum(labels).sum()
    dice = (2*intersection ) / (union + 1e-6)
    return dice

def dice_batch(outputs: np.array, labels: np.array):
    intersection = (outputs * labels).sum((1,2))
    union = np.sum(outputs).sum((1,2))+np.sum(labels).sum((1,2))
    dice = (2*intersection ) / (union + 1e-6)
    return dice.mean()

def get_region(mask_output_,ann_frame_idx_):
    for l in range(ann_frame_idx_):
        dice_scores_=dice(mask_output_[ann_frame_idx_-l],mask_output_[ann_frame_idx_])
        if dice_scores_==0:
            left_=ann_frame_idx_-l
            break
    for r in range(ann_frame_idx_,len(mask_output_)):
        dice_scores_=dice(mask_output_[r],mask_output_[ann_frame_idx_])
        if dice_scores_==0:
            right_=r
            break
    mask_output_[:left_]=mask_output_[:left_]*0
    mask_output_[right_:]=mask_output_[right_:]*0
    return mask_output_.astype(np.int16)

def dice_batch(outputs: np.array, labels: np.array):
    outputs=np.squeeze(outputs)
    labels=np.squeeze(labels)
    intersection = (outputs * labels).sum((1,2))
    union = outputs.sum((1,2))+labels.sum((1,2))
    dice = (2*intersection + 1e-8) / (union + 1e-8)
    return dice.mean()

In [744]:

    

def test_sam_func(pons,las,tres_,video_dir,ann_frame_idx_):
    predictor = build_sam2_video_predictor(model_cfg, checkpoint, device=device)
    inference_state = predictor.init_state(video_path=video_dir)
    predictor.reset_state(inference_state)

    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx_,
        obj_id=1,
        points=pons,
        labels=las,
    )
    
    abs_mask_region=np.sum((out_mask_logits[0] > 0.0).cpu().numpy())

    if abs_mask_region>tres_:
        return None, abs_mask_region
    else:
        video_segments_f = {} 
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
            video_segments_f[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state,reverse=True):
            video_segments_f[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }

        return video_segments_f, abs_mask_region
    
def dice_filter(results_,idx_):
    results_=np.squeeze(results_)
    imm_dim=results_.shape[1]
    final_mask=np.zeros((len(results_),imm_dim,imm_dim))

    keep=True
    for i in range(1,idx_):

        if dice(results_[idx_-i-1],results_[idx_-i])>0.5:
            keep=True
        else:
            keep=False
            final_mask[idx_-i]=results_[idx_-i]
            break
        if keep:
            final_mask[idx_-i]=results_[idx_-i]

    keep=True
    for i in range(idx_,len(results_)-1):
        
        if dice(results_[i+1],results_[i])>0.5:
            keep=True
        else:
            keep=False
            final_mask[i]=results_[i]
            break
        if keep:
            final_mask[i]=results_[i]
    return final_mask




In [981]:
def get_gmm_points(prompt,point_size,dist_pos_cata,dist_neg_cata,dist_gmm_pos,dist_gmm_neg):
    test_points=[]
    test_labels=[]
    if point_size==1:

        point_i=[]
        label_i=[]
        point_i.append(prompt[0].astype(np.int32))
        label_i.append(1.0)
        point_i.append(dist_gmm_neg[dist_neg_cata.sample((1,)).numpy()[0]].sample((1,)).numpy().astype(np.int32)[0])
        label_i.append(0.0)
        point_i=np.asarray(np.squeeze(point_i)).astype(np.float32)
        label_i=np.asarray(label_i).astype(np.int32)    
        test_points.append(point_i)
        test_labels.append(label_i)

    elif point_size>1:
        point_i=[]
        label_i=[]
        point_i.append(prompt[0])
        label_i.append(1.0)
        for j in range(point_size-1):
            point_i.append(dist_gmm_pos[dist_pos_cata.sample((1,)).numpy()[0]].sample((1,)).numpy().astype(np.int32)[0])
            label_i.append(1.0)
        for j in range(point_size):
            point_i.append(dist_gmm_neg[dist_neg_cata.sample((1,)).numpy()[0]].sample((1,)).numpy().astype(np.int32)[0])
            label_i.append(0.0)
        point_i=np.asarray(np.squeeze(point_i)).astype(np.float32)
        label_i=np.asarray(label_i).astype(np.int32)    
        test_points.append(point_i)
        test_labels.append(label_i)
    return test_points,test_labels

def get_score(results_,shape_,w_,h_):
    if results_==None:
        t1=1000
        t2=1000
        t3=1000
        mask_output_=np.ones(shape_)
    else:
        mask_output_=extract_mask(results_,w_,h_)
        t1,t2,t3=metrcis_func(mask_output_)
    return t1,t2,t3,mask_output_

def check_improvement(base_mask,mask_output_,k1_,k3_,t1_,t3_,mr,mr_,tres_mask):
    improvements=0
    improved=False
    if base_mask is not None:
        mask_compare=np.sum(np.abs(base_mask-mask_output_),axis=(1,2,3))

    else:
        mask_compare=np.ones((len(mask_output_)))*tres_mask
    
    imp_1=(k1_-t1_)/(k1_+1e-8) 
    imp_2=(k3_-t3_)/(k3_+1e-8)
    if imp_1>0 and imp_2>0 and mr<tres_mask:
        improvements=(imp_1 + imp_2)/2
    else:
        improvements=-np.inf 

    masks_diff_time=mask_compare>tres_mask

    add_rules=0
    if improvements>0.001 and np.sum(masks_diff_time)<=30.0 :
        print('New good points hit rule 1' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif improvements>0.1:
        print('New good points hit rule 2' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif t1_ !=1000 and (np.abs(imp_1)+np.abs(imp_2))/2<0.2:
        print('New good points hit rule 3' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif mr>tres_mask and mr_<tres_mask:
        improved=True
        print('New good points hit rule 4' )
        print('')
        print('')
        print('')
    elif mr_<mr:
        improved=True
        print('New good points hit rule 5' )
        print('')
        print('')
        print('')
    return improved,add_rules

In [ ]:
def get_input_and_mask(patient_num,org):
    f1, f2, f3, f4=get_data_path(patient_num)
    imp1,imp2,imp3=get_input_frame_all(f1,f2,f3,f4)
    mas_1=get_mask_all(org,f1[-1],f2[-1])
    mas_2=get_mask_all(org,f3[-1],f4[-1])
    input_and_mask_1=[imp1,mas_1]
    input_and_mask_2=[imp2,mas_1]
    input_and_mask_3=[imp3,mas_2]
    return input_and_mask_1,input_and_mask_2,input_and_mask_3



(35, 256, 256) (35, 256, 256)
(35, 256, 256) (35, 256, 256)
(36, 256, 256) (36, 256, 256)


# **save input img from dcm to jpg**

In [ ]:
base_path='/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/'
Jpg_folder_save_1='/T1DUAL/Inp_InPhase_jpg'
Jpg_folder_save_2='/T1DUAL/Inp_OutPhase_jpg'
Jpg_folder_save_3='/T2SPIR/Inp_jpg'

Ground_folder_t1='/T1DUAL/Ground'
Ground_folder_t2='/T2SPIR/Ground'


In [918]:
def get_data_path(pat_num):
    
    video_dirt_t1_t=base_path+patients[pat_num]+Ground_folder_t1
    video_dirt_t1_1=base_path+patients[pat_num]+dcm_files_in_t1
    video_dirt_t1_2=base_path+patients[pat_num]+dcm_files_out_t1
    _frame_names_t1_1 = [
            p for p in os.listdir(video_dirt_t1_1)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG", ".png",".dcm"]
        ]
    _frame_names_t1_2 = [
            p for p in os.listdir(video_dirt_t1_2)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG",".png",".dcm"]
        ]
    _frame_names_t1_t = [
            p for p in os.listdir(video_dirt_t1_t)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG",".png",".dcm"]
        ]
    
    path_t1=[video_dirt_t1_1, video_dirt_t1_2, video_dirt_t1_t]
    name_t1=[_frame_names_t1_1,_frame_names_t1_2,_frame_names_t1_t]

    video_dirt_t2_t=base_path+patients[pat_num]+Ground_folder_t2
    video_dirt_t2=base_path+patients[pat_num]+dcm_files_t2
    _frame_names_t2 = [
            p for p in os.listdir(video_dirt_t2)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG", ".png",".dcm"]
        ]
    _frame_names_t2_t = [
            p for p in os.listdir(video_dirt_t2_t)
            if os.path.splitext(p)[-1] in [".png"]#".jpg", ".jpeg", ".JPG", ".JPEG",,".dcm"
        ]
    
    path_t2=[video_dirt_t2, video_dirt_t2_t]
    name_t2=[_frame_names_t2,_frame_names_t2_t]
    return  path_t1,name_t1,path_t2,name_t2

In [474]:
def get_input_frame_all(path_1,name_1,path_2,name_2):
    input_t1_1=[]
    input_t1_2=[]
    for i in range(len(name_1[0])):
        ds_t1_1 = dcmread(os.path.join(path_1[0],name_1[0][i]))
        arr_t1_1 = ds_t1_1.pixel_array
        input_t1_1.append(arr_t1_1)
    print(os.path.join(path_1[0],name_1[0][i]))
    for i in range(len(name_1[1])):
        ds_t1_2 = dcmread(os.path.join(path_1[1],name_1[1][i]))
        arr_t1_2 = ds_t1_2.pixel_array
        input_t1_2.append(arr_t1_2)
    
    print(os.path.join(path_1[1],name_1[1][i]))
    input_t2=[]
    for i in range(len(name_2[0])):
        ds_t1_1 = dcmread(os.path.join(path_2[0],name_2[0][i]))
        arr_t1_1 = ds_t1_1.pixel_array
        input_t2.append(arr_t1_1)

    print(os.path.join(path_2[0],name_2[0][i]))
    return np.asarray(input_t1_1), np.asarray(input_t1_2), np.asarray(input_t2)

In [ ]:
def make_dirt(dirt):
    try:
        os.mkdir(dirt)
        print(f"Directory '{dirt}' created successfully.")
    except FileExistsError:
        print(f"Directory '{dirt}' already exists.")

In [555]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)
    kk1,kk2,kk3=get_input_frame_all(f1,f2,f3,f4)
    
    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_1
    make_dirt(directory_name)
    for j in range(len(f2[0])):#
        img_path=os.path.join(directory_name,f2[0][j][:-4])+'.png'
        zzz=kk1[j]/np.max(kk1[j])
        zzz=(zzz*255).astype(np.int32)
        image=Image.fromarray(zzz)
        image = np.asarray(image.convert("RGB"))#
        cv2.imwrite(img_path, image)
        image1 = Image.open(img_path)
        image1 = np.array(image1.convert("RGB"))
        print(np.sum(image1-image))
    
    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_2
    make_dirt(directory_name)
    for j in range(len(f2[1])):
        img_path=os.path.join(directory_name,f2[1][j][:-4])+'.png'
        image=Image.fromarray(kk2[j])
        zzz=kk2[j]/np.max(kk2[j])
        zzz=(zzz*255).astype(np.int32)
        image=Image.fromarray(zzz)
        image = np.asarray(image.convert("RGB"))# 
        cv2.imwrite(img_path, image)
        image1 = Image.open(img_path)
        image1 = np.array(image1.convert("RGB"))
        print(np.sum(image1-image))


    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_3
    make_dirt(directory_name)
    for j in range(len(f4[0])):
        img_path=os.path.join(directory_name,f4[0][j][:-4])+'.png'
        zzz=kk3[j]/np.max(kk3[j])
        zzz=(zzz*255).astype(np.int32)
        image=Image.fromarray(zzz)
        image = np.asarray(image.convert("RGB"))#

        cv2.imwrite(img_path, image)
        image1 = Image.open(img_path)
        image1 = np.array(image1.convert("RGB"))
        print(np.sum(image1-image))  

/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/DICOM_anon/InPhase/IMG-0004-00070.dcm
/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/DICOM_anon/OutPhase/IMG-0004-00069.dcm
/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/DICOM_anon/IMG-0002-00036.dcm
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_InPhase_jpg' already exists.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_OutPhase_jpg' created successfully.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/Inp_jpg' created successfully.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/DICOM_anon/InPhase/IMG-0010-00052.dcm
/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/DICOM_anon/OutPhase/IMG-0010-00051.dcm
/mnt/d/Dataset/data

In [611]:
f1,f2,f3,f4=get_data_path(0)

directory_name = base_path+'{}'.format(patients[0])+Jpg_folder_save_1
make_dirt(directory_name)
for j in range(3):#
    img_path_png=os.path.join(directory_name,f2[0][j][:-4])+'.png'
    img_path_jpg=os.path.join(directory_name,f2[0][j][:-4])+'.jpg'
    img_png=Image.open(img_path_png)
    img_png.save(img_path_jpg)

Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_InPhase_jpg' already exists.


In [612]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)

    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_1
    make_dirt(directory_name)
    for j in range(len(f2[0])):#
        img_path_png=os.path.join(directory_name,f2[0][j][:-4])+'.png'
        img_path_jpg=os.path.join(directory_name,f2[0][j][:-4])+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)

    
    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_2
    make_dirt(directory_name)
    for j in range(len(f2[1])):
        img_path_png=os.path.join(directory_name,f2[1][j][:-4])+'.png'
        img_path_jpg=os.path.join(directory_name,f2[1][j][:-4])+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)



    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_3
    make_dirt(directory_name)
    for j in range(len(f4[0])):
        img_path=os.path.join(directory_name,f4[0][j][:-4])+'.png'
        img_path_jpg=os.path.join(directory_name,f4[0][j][:-4])+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)


Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dat

In [632]:
directory_name

'/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/39/T2SPIR/Inp_jpg'

In [633]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)

    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_1
    make_dirt(directory_name)
    for j in range(len(f2[0])):#

        img_path_jpg=os.path.join(directory_name,f2[0][j][:-4])+'.jpg'
        os.rename(img_path_jpg,directory_name+'/{}'.format(j)+'.jpg')

    
    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_2
    make_dirt(directory_name)
    for j in range(len(f2[1])):

        img_path_jpg=os.path.join(directory_name,f2[1][j][:-4])+'.jpg'
        os.rename(img_path_jpg,directory_name+'/{}'.format(j)+'.jpg')



    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_3
    make_dirt(directory_name)
    for j in range(len(f4[0])):

        img_path_jpg=os.path.join(directory_name,f4[0][j][:-4])+'.jpg'
        os.rename(img_path_jpg,directory_name+'/{}'.format(j)+'.jpg')


Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T1DUAL/Inp_OutPhase_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T2SPIR/Inp_jpg' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T1DUAL/Inp_InPhase_jpg' already exists.
Directory '/mnt/d/Dat

In [937]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)

    directory_name = base_path+'{}'.format(patients[i])+Jpg_folder_save_3
    for j in range(len(f4[0])):
        img_path_png=os.path.join(directory_name,f4[0][j][:-4])+'.png'

        img_path_jpg=directory_name+'/{}'.format(j)+'.jpg'
        
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)
        # print(img_path_png)
        # print(img_path_jpg)

In [656]:
f1,f2,f3,f4=get_data_path(0)


In [932]:
Jpg_folder_save_3

'/T2SPIR/Inp_jpg'

In [659]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)

    directory_name = base_path+'{}'.format(patients[i])+Ground_folder_t1
    make_dirt(directory_name)
    for j in range(len(f2[2])):#
        img_path_png=os.path.join(directory_name,f2[2][j][:-4])+'.png'
        img_path_jpg=directory_name+'/{}'.format(j)+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)



    directory_name = base_path+'{}'.format(patients[i])+Ground_folder_t2
    make_dirt(directory_name)
    for j in range(len(f4[1])):
        img_path=os.path.join(directory_name,f4[1][j][:-4])+'.png'
        img_path_jpg=directory_name+'/{}'.format(j)+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)


Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T1DUAL/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T1DUAL/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T2SPIR/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T1DUAL/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T2SPIR/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T1DUAL/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T2SPIR/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/8/T1DUAL/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/8/T2SPIR/Ground' already exists.
Directory '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/10/T1DUAL/Ground' already exists

In [917]:
f4[1]

['0.jpg',
 '1.jpg',
 '10.jpg',
 '11.jpg',
 '12.jpg',
 '13.jpg',
 '14.jpg',
 '15.jpg',
 '16.jpg',
 '17.jpg',
 '18.jpg',
 '19.jpg',
 '2.jpg',
 '20.jpg',
 '21.jpg',
 '22.jpg',
 '23.jpg',
 '24.jpg',
 '25.jpg',
 '26.jpg',
 '27.jpg',
 '28.jpg',
 '29.jpg',
 '3.jpg',
 '30.jpg',
 '31.jpg',
 '32.jpg',
 '33.jpg',
 '34.jpg',
 '35.jpg',
 '4.jpg',
 '5.jpg',
 '6.jpg',
 '7.jpg',
 '8.jpg',
 '9.jpg',
 'IMG-0002-00001.png',
 'IMG-0002-00002.png',
 'IMG-0002-00003.png',
 'IMG-0002-00004.png',
 'IMG-0002-00005.png',
 'IMG-0002-00006.png',
 'IMG-0002-00007.png',
 'IMG-0002-00008.png',
 'IMG-0002-00009.png',
 'IMG-0002-00010.png',
 'IMG-0002-00011.png',
 'IMG-0002-00012.png',
 'IMG-0002-00013.png',
 'IMG-0002-00014.png',
 'IMG-0002-00015.png',
 'IMG-0002-00016.png',
 'IMG-0002-00017.png',
 'IMG-0002-00018.png',
 'IMG-0002-00019.png',
 'IMG-0002-00020.png',
 'IMG-0002-00021.png',
 'IMG-0002-00022.png',
 'IMG-0002-00023.png',
 'IMG-0002-00024.png',
 'IMG-0002-00025.png',
 'IMG-0002-00026.png',
 'IMG-0002-00027

In [930]:
for i in range(5):
    f1,f2,f3,f4=get_data_path(i)
    print(f3)

    # directory_name = base_path+'{}'.format(patients[i])+Ground_folder_t2

['/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/DICOM_anon/', '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/1/T2SPIR/Ground']
['/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T2SPIR/DICOM_anon/', '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/2/T2SPIR/Ground']
['/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T2SPIR/DICOM_anon/', '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/3/T2SPIR/Ground']
['/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T2SPIR/DICOM_anon/', '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/5/T2SPIR/Ground']
['/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/8/T2SPIR/DICOM_anon/', '/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/8/T2SPIR/Ground']


In [931]:
base_path+'{}'.format(patients[5])+Ground_folder_t2

'/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/10/T2SPIR/Ground'

In [921]:
for i in range(len(patients)):
    f1,f2,f3,f4=get_data_path(i)
    directory_name = base_path+'{}'.format(patients[i])+Ground_folder_t2
    for j in range(len(f4[1])):
        img_path_png=os.path.join(directory_name,f4[1][j][:-4])+'.png'
        img_path_jpg=directory_name+'/{}'.format(j)+'.jpg'
        img_png=Image.open(img_path_png)
        img_png.save(img_path_jpg)

# **main**

In [ ]:
Jpg_folder_save_1='/T1DUAL/Inp_InPhase_jpg'
Jpg_folder_save_2='/T1DUAL/Inp_OutPhase_jpg'    
Jpg_folder_save_3='/T2SPIR/Inp_jpg'

Ground_folder_t1='/T1DUAL/Ground'
Ground_folder_t2='/T2SPIR/Ground'


In [723]:
def get_data_path_jpg(pat_num,path_image,path_true):
    base_path='/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/'
    
    
    video_dirt_t1_t=base_path+patients[pat_num]+path_true
    video_dirt_t1_img=base_path+patients[pat_num]+path_image

    _frame_names_t1_img = [
            p for p in os.listdir(video_dirt_t1_img)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
        ]
    _frame_names_t1_img.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))

    _frame_names_t1_t = [
            p for p in os.listdir(video_dirt_t1_t)
            if os.path.splitext(p)[-1] in ['.png']#".jpg", ".jpeg", ".JPG", ".JPEG",
        ]
    # _frame_names_t1_t.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))

    path_t1=[video_dirt_t1_img , video_dirt_t1_t]
    name_t1=[_frame_names_t1_img,_frame_names_t1_t]

    return  path_t1,name_t1

In [905]:
def check_init_img(predictor_img,image,pons,las, mask_t):
    predictor_img.set_image(image)
    masks, scores, logits = predictor_img.predict(
    point_coords=pons,
    point_labels=las,
    multimask_output=True,)
    sorted_ind = np.argsort(scores)[::-1]
    masks = masks[sorted_ind]
    scores = scores[sorted_ind]
    logits = logits[sorted_ind]
    mask_p=(masks[0] > 0.0)
    return dice(np.squeeze(mask_p),np.squeeze(mask_t))

In [989]:
def __promt_idx(data_img,data_mask):
    masked_index=np.where(data_mask.sum((1,2))>0)[0].tolist()
    print(masked_index)
    ini_check=[]
    ini_check_ind=[]

    sam2_model = build_sam2(model_cfg, checkpoint, device=device)
    predictor_img = SAM2ImagePredictor(sam2_model)
    for i in range(3):
        index=masked_index.pop(np.random.randint(0,len(masked_index)))
        print(index)
        try:
            # print('1')
            point_r,point_l=get_contour(data_mask[index])
            # print('2')
            center_point = np.array((point_r,point_l), dtype=np.float32)
            points = np.array(center_point[np.newaxis,...], dtype=np.float32)
            labels = np.array([1,], np.int32)
            # print('3')
            zzz=check_init_img(predictor_img,data_img[index],points,labels,data_mask[index])
            # print('4')
            ini_check.append(zzz)
            print(zzz,index)
            # print('5')
            ini_check_ind.append(index)
        except:
            print("Some Error.")
    print(ini_check_ind[np.argmax(ini_check)])
    return ini_check_ind[np.argmax(ini_check)]


In [991]:

def test_and_val_gp(imps_,masks_,input_dir,_sample_=3):
    w,h=np.asarray(imps_[0]).shape[:2]
    region_cover=0.5
    Size_to_cover=(w+h)/2*region_cover
    mask_region_limit=np.asarray(w*h*region_cover).astype(np.int64)
    print('Image size is ',w*h*region_cover,mask_region_limit)
    mask_frame_idx=1000
    for i in range(10):
        try:
            mask_frame_idx=__promt_idx(imps_,masks_)
        except:
            print("Mask not right.")
        if mask_frame_idx!=1000:
            break
    print('Prompt Index is ',mask_frame_idx)
    point_r,point_l=get_contour(masks_[mask_frame_idx])
    center_point = np.array((point_r,point_l), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)

    decay=0.5
    num_of_gaus=6
    variance=np.sqrt(Size_to_cover/(num_of_gaus*3))
    wave_length=Size_to_cover/num_of_gaus
    rads_pos=np.arange(1,num_of_gaus+1)*wave_length
    rads_neg=rads_pos+3*variance
    alpha=[]
    for i in range(num_of_gaus):
        alpha.append((1-decay)**i)
    alpha=np.asarray(alpha).astype(np.float32)
    beta=alpha[::-1]

    cata_alpha = td.Categorical(torch.tensor(alpha/np.sum(alpha)))
    cata_beta = td.Categorical(torch.tensor(beta/np.sum(beta)))

    dist_pos=[]
    dist_neg=[]
    means_pos,means_neg=get_means((point_r,point_l),rads_pos,rads_neg,num_of_gaus)

    for i in range(len(means_pos)):
        gmm=Get_gmm(means_pos[i],variance)
        dist_pos.append(gmm)
        gmm=Get_gmm(means_neg[i],variance)
        dist_neg.append(gmm)
    


    center_point = np.array((point_r,point_l), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)

    results,mr_original=test_sam_func(points,labels,w*h,input_dir,mask_frame_idx)
    mask_output=extract_mask(results,w,h)
    filtered_output=dice_filter(mask_output,mask_frame_idx)

    # print(mask_output[0])

    mask_output_shape=mask_output.shape
    k1,k2,k3=metrcis_func(mask_output)
    print('           Initial metrics:                   ',k1,k2,k3,mr_original)
    
    mask_candidate=[]
    mask_candidate.append(dice_batch(mask_output,masks_))

    for i in range(_sample_):
        test_points,test_labels=get_gmm_points(points,1,cata_alpha,cata_beta,dist_pos,dist_neg)
        results_test,mr_test=test_sam_func(test_points,test_labels,mask_region_limit,input_dir,mask_frame_idx)  
        t1,t2,t3,mask_output_=get_score(results_test,mask_output_shape,w,h)     
        improved,add_rules=check_improvement(mask_output,mask_output_,k1,k3,t1,t3,mr_original,mr_test,mask_region_limit)
        print(t1,t2,t3,mr_test)
        if improved:
            k1=t1
            k3=t3
            filtered_output=dice_filter(mask_output_,mask_frame_idx)
            mask_candidate.append(dice_batch(filtered_output,masks_))
    for j in (2,3):
        for i in range(_sample_):
            test_points,test_labels=get_gmm_points(points,j,cata_alpha,cata_beta,dist_pos,dist_neg)
            results_test,mr_test=test_sam_func(test_points,test_labels,mask_region_limit,input_dir,mask_frame_idx)  
            t1,t2,t3,mask_output_=get_score(results_test,mask_output_shape,w,h)     
            print(t1,t2,t3,mr_test)
            improved,add_rules=check_improvement(mask_output,mask_output_,k1,k3,t1,t3,mr_original,mr_test,mask_region_limit)
            if improved :
                k1=t1
                k3=t3
                filtered_output=dice_filter(mask_output_,mask_frame_idx)
                mask_candidate.append(dice_batch(filtered_output,masks_))

    return mask_candidate



In [ ]:
reults_organ_0=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    kkk=test_and_val_gp(input_imgaes,mask_os,f1[0])
    reults_organ_0.append(np.max(kkk))
print(reults_organ_0)
print(np.mean(reults_organ_0))

In [825]:
np.mean(reults_organ_0)

np.float64(0.8789265797908324)

In [939]:
        
reults_organ_0_2=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    print(mask_os.shape,input_imgaes.shape)
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))

    for j in range(3):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_0_2.append(np.max(patients_i))


(36, 256, 256) (36, 256, 256, 3)
Image size is  32768.0 32768
[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
16
1
2
3
4
0.965870306837587 16
5
35
1
2
3
4
0.9247311821742398 35
5
24
1
2
3
4
0.011433868972275359 24
5
14
1
2
3
4
0.1180030255400862 14
5
20
1
2
3
4
0.2240459581375993 20
5
27
1
2
3
4
0.0067261973783223706 27
5
21
1
2
3
4
0.11366272323969781 21
5
32
1
2
3
4
0.9593476434873068 32
5
16
Prompt Index is  16


frame loading (JPEG): 100%|██████████| 36/36 [00:01<00:00, 31.79it/s]
/home/will/code/sam2-main/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/home/will/code/sam2-main/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 17/17 [00:01<00:00,  9.71it/s]


           Initial metrics:                    19.309303 10.521368 28.689835 24144


propagate in video: 100%|██████████| 17/17 [00:01<00:00,  9.73it/s]


New good points hit rule 1



5.3354216 4.5567985 13.293376 1422


...

10.800977 5.807723 14.536513 11632
New good points hit rule 3





propagate in video: 100%|██████████| 25/25 [00:02<00:00,  9.56it/s]


11.6511 8.125 22.101265 22522


propagate in video: 100%|██████████| 25/25 [00:02<00:00,  9.64it/s]

11.679206 8.156986 18.717022 22577
New good points hit rule 3





In [1022]:
reults_organ_0_2

[np.float64(0.9290477200418835),
 np.float64(0.8097504289568848),
 np.float64(0.9259282686651562),
 np.float64(0.7344956945848656),
 np.float64(0.9057712068816759),
 np.float64(0.8143624422761149),
 np.float64(0.8559410489512372),
 np.float64(0.8337877456759368),
 np.float64(0.8278364557976299),
 np.float64(0.8384936701280623),
 np.float64(0.8707592253059963),
 np.float64(0.9110935578168311),
 np.float64(0.944831093299206),
 np.float64(0.9225201559890375),
 np.float64(0.9146154316260554),
 np.float64(0.8311966901985649),
 np.float64(0.8265378810210425),
 np.float64(0.8728836637710207),
 np.float64(0.9471798919626814),
 np.float64(0.9228999065369613)]

In [1025]:
(np.mean(reults_organ_0_2)+np.mean(reults_organ_0))/2


np.float64(0.8754615943825872)

In [828]:
reults_organ_1=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(3):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_1.append(np.max(patients_i))


Image size is  32768.0 32768
0.848739492232187 10
0.9602272718179665 25
0.8956834516264686 11
0.8635983256371561 19
0.12283044042345735 15
0.9252259647286557 24
0.17654171683610434 21
0.9304897307026146 14
Prompt Index is  25


propagate in video: 100%|██████████| 26/26 [00:02<00:00,  9.42it/s]


           Initial metrics:                    20.933512 7.4509807 31.536146 18068


propagate in video: 100%|██████████| 26/26 [00:02<00:00,  9.44it/s]


New good points hit rule 1



11.486486 6.72093 17.75 508


...

5.1374106 4.5359983 4.9663653 10163
New good points hit rule 1





frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 33.56it/s]


1000 1000 1000 62022


propagate in video: 100%|██████████| 6/6 [00:00<00:00, 11.09it/s]

16.69079 6.818182 12.888 8591


In [829]:
print(reults_organ_1)
print(np.mean(reults_organ_1))

[np.float64(0.9352874401144805), np.float64(0.9671263037757796), np.float64(0.9384238416463323), np.float64(0.9590523210102384), np.float64(0.9160086701856716), np.float64(0.9437107927880711), np.float64(0.9703885734461153), np.float64(0.9464585054635445), np.float64(0.5524200982005852), np.float64(0.9339573200447201), np.float64(0.8551971000028056), np.float64(0.9069061519430445), np.float64(0.951182500563292), np.float64(0.9297952750325976), np.float64(0.9507889123863745), np.float64(0.9448172477462102), np.float64(0.9808698133525086), np.float64(0.9372663823981969), np.float64(0.942537007258033), np.float64(0.9465973738885498)]
0.9204395815623576


In [1026]:
reults_organ_1_2=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(4):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_1_2.append(np.max(patients_i))
    


Image size is  32768.0 32768
[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
15
0.965550238772464 15
13
0.9682539674001287 13
17
0.12962962952051377 17
13
Prompt Index is  13


frame loading (JPEG): 100%|██████████| 36/36 [00:01<00:00, 33.32it/s]
/home/will/code/sam2-main/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/home/will/code/sam2-main/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 14/14 [00:01<00:00,  9.87it/s]


           Initial metrics:                    5.4071827 3.4735153 11.608334 600


propagate in video: 100%|██████████| 14/14 [00:01<00:00,  9.85it/s]


6.8660765 3.9236684 8.266667 631


...

3.4086366 2.716988 5.2650604 849
New good points hit rule 1





propagate in video: 100%|██████████| 11/11 [00:01<00:00, 10.04it/s]


11.190813 5.620064 21.665844 19832


propagate in video: 100%|██████████| 11/11 [00:01<00:00, 10.15it/s]

7.074593 4.0036407 9.815978 1459


In [1028]:
print(reults_organ_1_2)
print(np.mean(reults_organ_1_2))

[np.float64(0.970650323960082), np.float64(0.9659135651520508), np.float64(0.954122507128761), np.float64(0.9142739900458156), np.float64(0.8941506996217317), np.float64(0.9727287613248321), np.float64(0.9483659909909198), np.float64(0.9473137950324534), np.float64(0.9830348211257658), np.float64(0.9007915698915845), np.float64(0.9190182609765168), np.float64(0.9627308299756022), np.float64(0.937076280509003), np.float64(0.9543867094658876), np.float64(0.9397070947768925), np.float64(0.9267496685454563), np.float64(0.9339268158960043), np.float64(0.9643108615088788), np.float64(0.9644866514620832), np.float64(0.9313798123428054)]
0.9442559504866562


In [1029]:
print((np.mean(reults_organ_1_2)+np.mean(reults_organ_1))/2)

0.932347766024507


In [ ]:
reults_organ_2=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(2,f1[1],f2[1])
    if i==7:
        print('erro at frame 13. Fixing...')
        mask_os[13][0:150,0:120]=0
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(3):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_2.append(np.max(patients_i))


erro at frame 13. Fixing...
Image size is  32768.0 32768
7
0.23999999969032257 7
9
0.2712933751090501 9
13
0.2583170251875567 13
10
0.8681996630015705 10
16
0.4406779648568419 16
14
0.14833759571823837 14
5
0.8934010137167495 5
12
0.1909353903655396 12
5
Prompt Index is  5


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 32.90it/s]
/home/will/code/sam2-main/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/home/will/code/sam2-main/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 6/6 [00:00<00:00, 11.32it/s]


           Initial metrics:                    5.088731 4.7426586 6.3178806 294


propagate in video: 100%|██████████| 6/6 [00:00<00:00, 11.16it/s]


8.5220995 6.7118225 5.539726 295


...

8.890209 4.486465 4.837255 13320


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 32.60it/s]


1000 1000 1000 53062


propagate in video: 100%|██████████| 6/6 [00:00<00:00, 11.12it/s]

9.812365 4.8694816 5.4240365 13797


In [866]:
reults_organ_2

[np.float64(0.9410172115436658),
 np.float64(0.9753169139339609),
 np.float64(0.9453037805373742),
 np.float64(0.36152148474310425),
 np.float64(0.8096906433649599),
 np.float64(0.9640187720999915),
 np.float64(0.942996080972076),
 np.float64(0.9291774559731576),
 np.float64(0.9018096950332952),
 np.float64(0.9039659264358354),
 np.float64(0.9090188682698255),
 np.float64(0.907954056018532),
 np.float64(0.9640230704720322),
 np.float64(0.9495103879896747),
 np.float64(0.9568849587585855),
 np.float64(0.9062168263020585),
 np.float64(0.9388530476353701),
 np.float64(0.9260942155695862),
 np.float64(0.8213346768447295),
 np.float64(0.8906314482609622)]

In [ ]:

np.mean(reults_organ_2)

np.float64(0.8922669760379389)

In [1030]:
reults_organ_2_2=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(2,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(4):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_2_2.append(np.max(patients_i))


Image size is  32768.0 32768
[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
15
0.9303797458540298 15
21
0.9067103104709708 21
28
0.6136363632261789 28
15
Prompt Index is  15


propagate in video: 100%|██████████| 16/16 [00:01<00:00,  9.83it/s]


           Initial metrics:                    10.014085 6.8902717 14.408377 506


propagate in video: 100%|██████████| 16/16 [00:01<00:00,  9.62it/s]


New good points hit rule 5



10.616264 6.901768 9.330961 505


...

1000 1000 1000 57487


propagate in video: 100%|██████████| 7/7 [00:00<00:00, 10.96it/s]


8.495748 5.0642204 5.0669274 18066


propagate in video: 100%|██████████| 7/7 [00:00<00:00, 10.85it/s]

9.632208 4.620784 15.885563 17537


In [1031]:
print(reults_organ_2_2)
print(np.mean(reults_organ_2_2))

[np.float64(0.9353882348731695), np.float64(0.717067802504991), np.float64(0.9474215022710806), np.float64(0.9452161531696189), np.float64(0.8926106377479951), np.float64(0.977206318597146), np.float64(0.9376180877806902), np.float64(0.9293671306934376), np.float64(0.9755981315756759), np.float64(0.9634654972295376), np.float64(0.9304516088575652), np.float64(0.9563878106155245), np.float64(0.9638996561491544), np.float64(0.9408424975133308), np.float64(0.9346673969941082), np.float64(0.8820182596701038), np.float64(0.9599999045238252), np.float64(0.7658815263409375), np.float64(0.9751049337532), np.float64(0.9665913899146212)]
0.9248402240387857


In [1032]:
print((np.mean(reults_organ_2_2)+np.mean(reults_organ_2))/2)

0.9085536000383623


In [872]:
reults_organ_3=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(2):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_3.append(np.max(patients_i))


Image size is  32768.0 32768
18
0.015241524151500659 18
20
0.02594244020928007 20
32
0.8335451069459402 32
17
0.010000617321439557 17
27
0.04681459513600516 27
33
0.9661704993691874 33
31
0.04474769914076231 31
29
0.9118852449673307 29
33
Prompt Index is  33


frame loading (JPEG): 100%|██████████| 35/35 [00:01<00:00, 32.67it/s]
/home/will/code/sam2-main/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/home/will/code/sam2-main/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 34/34 [00:03<00:00,  9.42it/s]


           Initial metrics:                    8.692041 7.134663 15.103225 388


propagate in video: 100%|██████████| 34/34 [00:03<00:00,  9.43it/s]


New good points hit rule 3



9.024476 6.7557254 15.625805 389


...

propagate in video: 100%|██████████| 13/13 [00:01<00:00,  9.91it/s]


10.524519 5.08441 17.985714 1964


propagate in video: 100%|██████████| 13/13 [00:01<00:00,  9.85it/s]


4.5916696 3.9906056 5.1652617 2020
New good points hit rule 1



Image size is  41472.0 41472
22
0.8234872893578635 22
18
0.011436286383217397 18
23
0.09464893287642544 23
24
0.36783733792251627 24
19
0.03802281368678151 19
20
0.07024348200566533 20
21
0.07497189432001926 21
Mask not right.
19
0.03802281368678151 19
23
0.09464893287642544 23
24
0.36783733792251627 24
21
0.07497189432001926 21
20
0.07024348200566533 20
18
0.011436286383217397 18
22
0.8234872893578635 22
Mask not right.
21
0.07497189432001926 21
18
0.011436286383217397 18
22
0.8234872893578635 22
24
0.36783733792251627 24
19
0.03802281368678151 19
23
0.09464893287642544 23
20
0.07024348200566533 20
Mask not right.
18
0.011436286383217397 18
20
0.07024348200566533 20
19
0.03802281368678151 19
22
0.8234872893578635 22
23
0.09464893287642544 23
21
0.07497189432001926 21
24
0.36783733792251627 24
Mask not right.
22
0.8234872893578635 22
21
0.07497189432001926 21
19
0.03802281368678151 19
24
0.36783733792251627 24
23
0.094648

IndexError: index 1000 is out of bounds for axis 0 with size 34

In [875]:
np.mean(reults_organ_3)

np.float64(0.9216742790175179)

In [881]:
print(reults_organ_3)
np.mean(reults_organ_3)

[np.float64(0.8837780640345764), np.float64(0.8977048603430563), np.float64(0.9324134128266183), np.float64(0.9016647168360818), np.float64(0.9420302581243695), np.float64(0.8808063605516208), np.float64(0.9049627417465946), np.float64(0.8674994383056607), np.float64(0.9308382127760488), np.float64(0.9279521025398648), np.float64(0.9407323753235116), np.float64(0.939994773965481), np.float64(0.9392974230066312), np.float64(0.9311222822428518), np.float64(0.9358614876976068), np.float64(0.9075746103322282), np.float64(0.9534357593324209), np.float64(0.9724681423300983), np.float64(0.8963515607692661)]


np.float64(0.920341504372873)

In [1036]:
reults_organ_3_2=[]

for i in range(len(patients)):
    patients_i=[]
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0]) 
    for j in range(4):
        kkk=test_and_val_gp(input_imgaes,mask_os,f1[0],2)
        patients_i.append(np.max(kkk))
    reults_organ_3_2.append(np.max(patients_i))



Image size is  32768.0 32768
[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
23
0.9458858406627977 23
25
0.558139534659928 25
26
0.9675324669041996 26
26
Prompt Index is  26


frame loading (JPEG): 100%|██████████| 36/36 [00:01<00:00, 33.66it/s]
/home/will/code/sam2-main/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/home/will/code/sam2-main/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 27/27 [00:02<00:00,  9.37it/s]


           Initial metrics:                    4.578 2.9439998 3.2057521 760


propagate in video: 100%|██████████| 27/27 [00:02<00:00,  9.48it/s]


New good points hit rule 3



5.1961594 3.6323729 3.5146344 750


...

10.51361 4.858548 12.18455 2208


propagate in video: 100%|██████████| 17/17 [00:01<00:00,  9.61it/s]


8.834341 7.2999783 11.674731 16037


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 34.78it/s]


1000 1000 1000 44994


In [1037]:
print(reults_organ_3_2)
print(np.mean(reults_organ_3_2))

[np.float64(0.9762914711392381), np.float64(0.6647727858210017), np.float64(0.9856636418409491), np.float64(0.9451326309786567), np.float64(0.979640143986777), np.float64(0.933910292158255), np.float64(0.9764158788286361), np.float64(0.8480408844721042), np.float64(0.9072511318899644), np.float64(0.6095653596243119), np.float64(0.9341119879593608), np.float64(0.9822097347519623), np.float64(0.8932777147949632), np.float64(0.9469722621285194), np.float64(0.971915501559902), np.float64(0.937421081779151), np.float64(0.960357482164872), np.float64(0.9470216891718456), np.float64(0.9176245125581088), np.float64(0.6688395057426926)]
0.8993217846675636


In [1039]:
print((np.mean(reults_organ_3_2)+np.mean(reults_organ_3))/2)

0.9098316445202184
